# AI Project : Simplified Wumpus World and First Order Logic

We based our project on the aima 2017 library. We have used the following files :
* agent.py - contains the wumpus world environment
* logic.py - contains databases and solvers
* planning.py - contains first order logic algorithm


In this file we are using first order logic, we use the graph solver from the aima library. Nevertheless, the library is not complete. For example it doesn't implement the followings for the first order logic :
* <=>
* ~
* forall(x)
* thereexist(x)

The first order solver used ask something to the knowledge base is also very slow. 
In this file we are using a very simplified version of the wumpus world. We assume that the position of the gold and the obstacles are known and use the graphlan algorithm to find the best path.

# Generate and draw the wumpus world

In this part we generate a wumpus world.

In [1]:
from ipythonblocks import BlockGrid
from IPython.display import clear_output
from agents import *
import time

color = {"Breeze": (225, 225, 225),
        "Pit": (0,0,0),
        "Gold": (253, 208, 23),
        "Glitter": (253, 208, 23),
        "Wumpus": (255, 0, 255),
        "Stench": (128, 0, 128),
        "Explorer": (0, 0, 255),
        "Wall": (255, 0, 0)
        }

def program(percepts):
    '''Returns an action based on it's percepts'''
    print(percepts)
    return input()

pit_proba = 0.05
w = WumpusEnvironment(program, 7,7,pit_proba)         
grid = BlockGrid(w.width, w.height, fill=(123, 234, 123))

def draw_grid(world):
    global grid
    grid[:] = (123, 234, 123)
    for x in range(0, len(world)):
        for y in range(0, len(world[x])):
            if len(world[x][y]):
                g = -1
                for i in world[x][y]:
                    h = world[x][y].index(i)
                    if(isinstance(i, Gold)):
                        g = world[x][y].index(i)
                        break
                    elif(isinstance(i, Wumpus)):
                        g = h
                    elif(isinstance(i, Pit)):
                        if (not isinstance(world[x][y][g],Wumpus)):
                            g = h

                grid[y, x] = color[world[x][y][g].__class__.__name__]

## First order logic

We connect the tiles together, assume safe everything that is not a pit or a wumpus, take the gold as a goal and define the action move.
We tried to implement a more complicated world but the solver was taking several minutes to answer. We decided then decided to switch to porositional logic to solve the full problem

In [2]:
from ipythonblocks import BlockGrid
from agents import *
from utils import *
from planning import *
from logic import *

world = w.get_world()

xmax=len(world)
ymax=len(world[0])

knowledge_base = []

############ DEFINE FOL #############################
# Initial State of the World
for i in range(0,xmax):
        for j in range(0,ymax):
                unsafe = False;
                for h in range(0,len(world[i][j])):
                        if(isinstance(world[i][j][h],Wumpus) | isinstance(world[i][j][h],Pit) | isinstance(world[i][j][h],Wall)):
                                unsafe = True;
                        elif(isinstance(world[i][j][h],Gold)):
                                goal_state = expr("At(T_%do%d)"%(i,j));
                if(unsafe==False):           
                        knowledge_base.append(expr("Safe(T_%do%d)"%(i,j)));
                                     
for i in range(0,xmax):
        for j in range(0,ymax):
            if(i==0 & j!=0):
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i+1,j)));       
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i,j+1))); 
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i,j-1)));  
            elif(j==0 & i!=0):
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i+1,j)));       
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i-1,j)));  
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i,j+1))); 
            elif(i==0 & j==0):
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i+1,j)));         
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i,j+1)));                 
            else:
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i+1,j)));       
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i-1,j)));  
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i,j+1))); 
                knowledge_base.append(expr("Connected(T_%do%d,T_%do%d)"%(i,j,i,j-1)));  


#Initial State of the Explorer
knowledge_base.append(expr("At(T_1o1)"))


    
##########  DEFINE ACTIONS ###########################################

#Move to a safe position
precond_pos = [expr('At(x)'),expr("Safe(y)"),expr("Connected(x,y)")]
precond_neg = []
effect_add = [expr('At(y)')]
effect_rem = [expr('At(x)')]
move = Action(expr('Move(x, y)'), [precond_pos, precond_neg], [effect_add, effect_rem])

pddl = PDDL(knowledge_base, [move], goal_state)

########## USE GRAPHPLAN ############################################
negkb = FolKB([])
graphplan = GraphPlan(pddl, negkb)


def goal_test(kb, goals):
        return all(kb.ask(q) is not False for q in goals)

def run_graphplan():
    goals_pos = [goal_state]
    goals_neg = []

    while True:
        if (goal_test(graphplan.graph.levels[-1].poskb, goals_pos) and
                graphplan.graph.non_mutex_goals(goals_pos+goals_neg, -1)):
            solution = graphplan.extract_solution(goals_pos, goals_neg, -1)
            if solution:
                return solution
        graphplan.graph.expand_graph()
        if len(graphplan.graph.levels)>=2 and graphplan.check_leveloff():
            return None
        
solution = run_graphplan()

######## EXTRACT THE FINAL SOLUTION FROM GRAPHPLAN #################
if solution == None:
    final_solution = None;
else :
    final_solution = []
    s=solution[0];
    for i in range(0,len(s)):
        for j in range(0,len(s[i])):
            if("Move" in repr(s[i][j])):
                final_solution.append(s[i][j])
                
print("Solution of the problem : ", final_solution)


Solution of the problem :  [Move(T_1o1, T_1o2), Move(T_1o2, T_1o3), Move(T_1o3, T_2o3), Move(T_1o1, T_1o2), Move(T_1o2, T_2o2), Move(T_1o1, T_2o1), Move(T_2o1, T_2o2), Move(T_2o2, T_2o3), Move(T_2o3, T_3o3), Move(T_1o1, T_1o2), Move(T_1o2, T_2o2), Move(T_1o1, T_2o1), Move(T_2o1, T_2o2), Move(T_2o2, T_3o2), Move(T_1o1, T_2o1), Move(T_2o1, T_3o1), Move(T_3o1, T_3o2), Move(T_3o2, T_3o3), Move(T_3o3, T_4o3), Move(T_1o1, T_2o1), Move(T_2o1, T_3o1), Move(T_3o1, T_4o1), Move(T_4o1, T_4o2), Move(T_1o1, T_1o2), Move(T_1o2, T_2o2), Move(T_1o1, T_2o1), Move(T_2o1, T_2o2), Move(T_2o2, T_3o2), Move(T_1o1, T_2o1), Move(T_2o1, T_3o1), Move(T_3o1, T_3o2), Move(T_3o2, T_4o2), Move(T_4o2, T_4o3), Move(T_4o3, T_5o3), Move(T_5o3, T_5o4), Move(T_1o1, T_1o2), Move(T_1o2, T_1o3), Move(T_1o3, T_2o3), Move(T_1o1, T_1o2), Move(T_1o2, T_2o2), Move(T_1o1, T_2o1), Move(T_2o1, T_2o2), Move(T_2o2, T_2o3), Move(T_2o3, T_3o3), Move(T_1o1, T_1o2), Move(T_1o2, T_2o2), Move(T_1o1, T_2o1), Move(T_2o1, T_2o2), Move(T_2o2, 

##  Animation

Play the solution, check that there exists a solution (final_solution != None)


In [3]:
#Convert the final solution into a list of numbers used for the animation

if(final_solution!=None) :
    goal_state_string=repr(goal_state).split('_')[1].split(')')[0]
    list_of_action =[]
    j=1
    for i in range(0,len(final_solution)):
        action = final_solution[i]
        if((repr(action).split('_')[2].split(')')[0])==goal_state_string):
            while(repr(final_solution[i-j]).split('_')[1].split(',')[0]!="1o1"):
                  j=j+1
            for h in range(i-j,i+1):
                list_of_action.append(repr(final_solution[h]))
                
    final_list_of_action = []
    for x in list_of_action :
        final_list_of_action.append(x.split('_')[1].split(',')[0].split('o')[0])
        final_list_of_action.append(x.split('_')[1].split(',')[0].split('o')[1])
    final_list_of_action.append(list_of_action[len(list_of_action)-1].split('_')[2].split(')')[0].split('o')[0])
    final_list_of_action.append(list_of_action[len(list_of_action)-1].split('_')[2].split(')')[0].split('o')[1])

    print(final_list_of_action)             
        
            

['1', '1', '2', '1', '3', '1', '3', '2', '4', '2', '4', '3', '4', '4', '5', '4', '5', '5']


In [4]:
#Grid printing function
def animateGrid(acts):
    draw_grid(w.get_world())
    grid.show()
    for act in acts:
        print(act)
        w.step([act])
        for block in grid:
            clear_output() 
        draw_grid(w.get_world())
        grid.show()
        sleep(1)

In [5]:
from animation import *
if(final_solution!=None) :
    acts = getActs(final_list_of_action)
    animateGrid(acts)

,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
